## Importing libraries

In [1]:
import pandas as pd
import urllib.request
from matplotlib import pyplot as plt
import cv2
import os.path
import numpy as np
import threading

## Setting the img scrapper

In [2]:
opener=urllib.request.build_opener()
opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

In [3]:
df = pd.read_csv('public_Card_YGO.csv')
display(df)

,id,id_card,name,type,description,atk,def,level,race,attribute,...,set_rarity_code,set_price,image_url,name_en,image_url_yugipedia,image_url_cm,ban_tcg,collectionId,property_card,set_rarity_cm
0,369,17607,Piège des Protecteurs du Tombeau,Trap,"Tant que ""Permutation des Mondes"" est dans vot...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Gravekeeper's Trap,NaN,NaN,Unlimited,NaN,Continuous,Ultra Parallel Rare
1,2554,4986,Cadeau de l'Elfe Mystique,Trap,Augmentez vos Life Points de 300 points pour c...,NaN,NaN,NaN,NaN,NaN,...,SP,NaN,https://images.ygoprodeck.com/images/cards/982...,Gift of The Mystical Elf,NaN,NaN,Unlimited,NaN,Normal,Common
2,1211,8330,Fauchon β,Monster,Si cette carte détruit un monstre de l'adversa...,1200,800,4.0,Machine / Effect,LIGHT,...,R,NaN,https://images.ygoprodeck.com/images/cards/861...,Falchionβ,NaN,NaN,Unlimited,NaN,NaN,NaN
3,2207,7331,Moteur Démoniaque Oméga,Monster,Durant votre End Phase : Invoquez Spécialement...,2800,2000,8.0,Machine / Effect,DARK,...,ScR,NaN,https://images.ygoprodeck.com/images/cards/825...,Fiendish Engine Ω,NaN,NaN,Unlimited,NaN,NaN,NaN
4,28505,17994,Hydrant des Secours-AS,Monster,"Tant que vous contrôlez un monstre ""Secours-AS...",0,0,1.0,Machine / Effect,FIRE,...,CR,NaN,NaN,Rescue-ACE Hydrant,NaN,NaN,Unlimited,NaN,NaN,Collectors Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28568,13206,8475,Dragon des Étoiles Majestueux,Monster,"""Dragon Majestueux"" + ""Dragon Poussière d'Étoi...",3800,3000,10.0,Dragon / Synchro / Effect,WIND,...,R,NaN,https://images.ygoprodeck.com/images/cards/784...,Majestic Star Dragon,NaN,NaN,Unlimited,NaN,NaN,NaN
28569,13278,5678,Frappe Destructrice Continue,Spell,Quand l'ATK d'un monstre attaquant sur le terr...,NaN,NaN,NaN,NaN,NaN,...,C,NaN,https://images.ygoprodeck.com/images/cards/680...,Continuous Destruction Punch,NaN,NaN,Unlimited,NaN,Continuous,NaN
28570,13288,5128,Impact de Météores Féeriques,Spell,Si le monstre équipé attaque un monstre en Pos...,NaN,NaN,NaN,NaN,NaN,...,SR,NaN,https://images.ygoprodeck.com/images/cards/976...,Fairy Meteor Crush,NaN,NaN,Unlimited,NaN,Equip,NaN
28571,13340,4814,Côntrole Hypnotique,Spell,Durant le prochain tour de votre adversaire ap...,NaN,NaN,NaN,NaN,NaN,...,UR,NaN,https://images.ygoprodeck.com/images/cards/486...,Mesmeric Control,NaN,NaN,Unlimited,NaN,Normal,NaN


## We remove the columns we dont want to use 

In [4]:
col_names = list(df.columns)
list_to_remove = ['name', 'set_code', 'image_url', 'id_card','id']
final_list = list(set(col_names) - set(list_to_remove))

In [5]:
df_name_url_setcode = df.drop(final_list, axis = 1,inplace= False)

In [6]:
display(df_name_url_setcode)

,id,id_card,name,set_code,image_url
0,369,17607,Piège des Protecteurs du Tombeau,MAMA-FR029,NaN
1,2554,4986,Cadeau de l'Elfe Mystique,SDP-FR009,https://images.ygoprodeck.com/images/cards/982...
2,1211,8330,Fauchon β,ANPR-FR039,https://images.ygoprodeck.com/images/cards/861...
3,2207,7331,Moteur Démoniaque Oméga,TDGS-FR095,https://images.ygoprodeck.com/images/cards/825...
4,28505,17994,Hydrant des Secours-AS,AMDE-FR004,NaN
...,...,...,...,...,...
28568,13206,8475,Dragon des Étoiles Majestueux,DP10-FR017,https://images.ygoprodeck.com/images/cards/784...
28569,13278,5678,Frappe Destructrice Continue,DR1-FR140,https://images.ygoprodeck.com/images/cards/680...
28570,13288,5128,Impact de Météores Féeriques,SDP-FR063,https://images.ygoprodeck.com/images/cards/976...
28571,13340,4814,Côntrole Hypnotique,SOI-FRSE4,https://images.ygoprodeck.com/images/cards/486...


## Print info about the Nan Values

In [7]:
num_image_url_Nan = df['image_url'].isna().sum()
num_image_url_yugipedia_Nan = df['image_url_yugipedia'].isna().sum()
num_image_url_cm_Nan = df['image_url_cm'].isna().sum()

print('num_image_url_Nan: ',num_image_url_Nan,', image_url_yugipedia :', num_image_url_yugipedia_Nan,', num_image_url_cm_Nan',num_image_url_cm_Nan  )


num_image_url_Nan:  17 , image_url_yugipedia : 28573 , num_image_url_cm_Nan 28573


## We drop the nan values of the csv file, then we reset the index

In [8]:
df_name_url_setcode =df_name_url_setcode.dropna(subset=['image_url'])
df_name_url_setcode.reset_index(drop=True, inplace=True)

In [9]:
display(df_name_url_setcode)

,id,id_card,name,set_code,image_url
0,2554,4986,Cadeau de l'Elfe Mystique,SDP-FR009,https://images.ygoprodeck.com/images/cards/982...
1,1211,8330,Fauchon β,ANPR-FR039,https://images.ygoprodeck.com/images/cards/861...
2,2207,7331,Moteur Démoniaque Oméga,TDGS-FR095,https://images.ygoprodeck.com/images/cards/825...
3,2555,4986,Cadeau de l'Elfe Mystique,SKE-FR044,https://images.ygoprodeck.com/images/cards/982...
4,5467,15238,Soldat Gaïa le Chevalier Implacable,ROTD-FR004,https://images.ygoprodeck.com/images/cards/333...
...,...,...,...,...,...
28551,13206,8475,Dragon des Étoiles Majestueux,DP10-FR017,https://images.ygoprodeck.com/images/cards/784...
28552,13278,5678,Frappe Destructrice Continue,DR1-FR140,https://images.ygoprodeck.com/images/cards/680...
28553,13288,5128,Impact de Météores Féeriques,SDP-FR063,https://images.ygoprodeck.com/images/cards/976...
28554,13340,4814,Côntrole Hypnotique,SOI-FRSE4,https://images.ygoprodeck.com/images/cards/486...


## We divide the  dataframe into 4 differents parts 

In [10]:
quarters = np.array_split(df_name_url_setcode, 4)


## Create the downloader class

In [11]:
class Picture_downloader:
    
    def download_pictures(self):
        
        for index in self.dataframe.index:
    
            url = df_name_url_setcode["image_url"][index]
            name = df_name_url_setcode["name"][index]
            id_card = df_name_url_setcode["id_card"][index]
            set_code = df_name_url_setcode["set_code"][index]
            id = df_name_url_setcode["id"][index]

            folder_path = os.path.join(self.base_folder_name,str(id))
            img_path = os.path.join(folder_path, str(id_card)+'.jpg')
            
            try:
                os.mkdir(folder_path)
                try:
                    urllib.request.urlretrieve(url, img_path) 
                except Exeption as e :
                    print(e)
                    print("error with :", name, "cant write")
                    print("url :", url)
                    
            except:
                print("folder :", folder_path, "- can't be created")
                
                

        
    def __init__(self, dataframe, base_folder_name ='Dataset_YGO_multi' ):
        self.dataframe = dataframe
        self.base_folder_name = base_folder_name
        print("size of the dataframe :", len(self.dataframe))
        t = threading.Thread(target=self.download_pictures)
        t.start()

In [12]:
Picture_downloader(quarters[0])
Picture_downloader(quarters[1])
Picture_downloader(quarters[2])
Picture_downloader(quarters[3])

size of the dataframe : 7139
size of the dataframe : 7139
size of the dataframe : 7139
size of the dataframe : 7139
